In [2]:
pip install split-folders

Defaulting to user installation because normal site-packages is not writeable
  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.0.0+cu118
Uninstalling torch-2.0.0+cu118:
  Successfully uninstalled torch-2.0.0+cu118
Found existing installation: torchvision 0.15.1+cu118
Uninstalling torchvision-0.15.1+cu118:
  Successfully uninstalled torchvision-0.15.1+cu118
Found existing installation: torchaudio 2.0.1+cu118
Uninstalling torchaudio-2.0.1+cu118:
  Successfully uninstalled torchaudio-2.0.1+cu118
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 953.7 kB/s eta 0:00:00 eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 28.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 22.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached https://download.pytorch.org/whl/filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.0-py3-none-any.whl (2.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 14.8 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached https://download.pytorch.org/whl/cmake-3.25.0-py2.py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (23.7 MB)
  Using cached https://download.pytorch.org/whl/mpmath-1.2.1-py3-none-a

In [2]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import splitfolders
from datetime import datetime

In [3]:
torch.version.cuda

'11.7'

In [2]:
input_folder ='./Bottle Images'
splitfolders.ratio(input_folder, output = "splitted_bootles", seed = 42, ratio = (.7, .2, .1), group_prefix = None)

In [3]:
training_dataset_path = './splitted_bootles/train'

In [4]:
training_transforms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])

In [5]:
training_dataset = torchvision.datasets.ImageFolder(root = training_dataset_path, transform = training_transforms)

In [6]:
training_loader = torch.utils.data.DataLoader(dataset = training_dataset, batch_size = 32, shuffle = False)

In [7]:
def get_mean_and_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += image_count_in_a_batch
        
    mean /= total_images_count
    std /= total_images_count
    
    return mean, std

In [8]:
mean_and_std = get_mean_and_std(training_loader)
print(mean_and_std)

(tensor([0.4729, 0.4099, 0.3521]), tensor([0.1786, 0.1670, 0.1610]))


In [9]:
mean_and_std[0]

tensor([0.4729, 0.4099, 0.3521])

In [10]:
mean_and_std[1]

tensor([0.1786, 0.1670, 0.1610])

In [19]:
mean_and_std = (torch.Tensor([0.4729, 0.4099, 0.3521]), torch.Tensor([0.1786, 0.1670, 0.1610]))

In [20]:
training_dataset_path = './splitted_bootles/train'
validation_dataset_path = './splitted_bootles/val'

In [21]:
training_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean_and_std[0], mean_and_std[1])
])

validation_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean_and_std[0], mean_and_std[1])
])

In [22]:
training_dataset = torchvision.datasets.ImageFolder(root = training_dataset_path, transform = training_transforms)
validation_dataset = torchvision.datasets.ImageFolder(root = validation_dataset_path, transform = validation_transforms)

In [23]:
training_loader = torch.utils.data.DataLoader(dataset = training_dataset, batch_size = 32, shuffle = True)
validation_loader = torch.utils.data.DataLoader(dataset = validation_dataset, batch_size = 32, shuffle = False)

In [43]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda"
    else:
        dev = "cpu"
    return torch.device(dev)

In [44]:
def train_nn(model, best_acc, training_loader, validation_loader, criterion, optimizer, n_epochs):
    device = set_device()
    
    for epoch in range(n_epochs):
        print("Epoch number %d" % (epoch + 1))
        model.train()
        running_loss = 0.
        running_correct = 0.
        total = 0
        
        for data in training_loader:
           
            images, labels = data
            
            images = images.to(device)
           
            labels = labels.to(device)
            
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
           
            _, predicted = torch.max(outputs.data, 1)
        
            loss = criterion(outputs, labels)
          
            loss.backward()
           
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
            
        
        epoch_loss = running_loss/len(training_loader)
        epoch_acc = 100.0 * running_correct / total
        
        print("    - Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f"
             % (running_correct, total, epoch_acc, epoch_loss))
        
        test_dataset_acc = evaluate_model_on_validation_set(model, validation_loader)
        
        if(test_dataset_acc > best_acc):
            best_acc = test_dataset_acc
            save_best_state(model, epoch, optimizer, best_acc)
        
    print("    Finished")
    

In [45]:
def evaluate_model_on_validation_set(model, validation_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    
    with torch.no_grad():
        for data in validation_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predicted_correctly_on_epoch += (labels == predicted).sum().item()
    epoch_acc = 100.0 * predicted_correctly_on_epoch / total
    print("    - Testing dataset. Got %d out of %d images correctly (%.3f%%)"
         % (predicted_correctly_on_epoch, total, epoch_acc))
    return epoch_acc
    

In [46]:
def save_best_state(model, epoch, optimizer, best_acc):
    state = {
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'best_accuracy': best_acc,
        'optimizer': optimizer.state_dict(),
        'date': datetime.now()
    }
    torch.save(state, 'best_model.pth.tar')

In [47]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(weights=None)
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, 5) # 5 - number of classes
device = set_device()
resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18_model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 0.003)

In [48]:
acc = 0.
n_epoch = 1
if (torch.load('best_model.pth.tar')):
    best = torch.load('best_model.pth.tar')
    acc = best['best_accuracy']
    optimizer.load_state_dict(best['optimizer'])
    resnet18_model.load_state_dict(best['model'])
    
print(acc)

train_nn(resnet18_model, acc, training_loader, validation_loader, loss_fn, optimizer, n_epoch)
best = torch.load('best_model.pth.tar')
print(best['epoch'])
print(best['best_accuracy'])
print(best['date'])
resnet18_model.load_state_dict(best['model'])
torch.save(resnet18_model, 'best_model.pth')

98.22
Epoch number 1
    - Training dataset. Got 16877 out of 17500 images correctly (96.440%). Epoch loss: 0.115
    - Testing dataset. Got 4833 out of 5000 images correctly (96.660%)
    Finished
5
98.22
2023-06-06 15:33:03.755987
